# Decition Tree part one, D3

## First steps
The first steps is to do all the imports and example data

In [322]:
import numpy as np
import pandas as pd
import random as rnd

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## Math
Using the D3 algorithm there is a lot of mathematical formulas we need. Theese includes entropy and information gain.

In [116]:
#Takes a list of probabilities and returns the entropy
def entropy(xs):
    return sum([-p*np.log2(p) for p in xs])

#Takes a list of an attribute and returns its entropy
def entropy_el(xs):
    return entropy(probability_el(xs))

def probability_el(xs):
    return [x / sum(xs) for x in xs]

In [314]:
#Takes a list of attributes, and counts them into dictionary
def labels(xs):
    lbs = {}
    for x in xs:
        if x in lbs:
            lbs[x] += 1
        else:
            lbs[x] = 1
    return lbs

#Returns entropy of a list with lables
def labels_entropy(xs):
    return entropy_el(labels(np.asarray(xs)).values())

#Takes two lists of lables and returns entropy
def entropy_given(xs, ys):
    labels_x = labels(xs)
    probs_x = probability_el(labels_x.values())
    total = 0
    for lab, prob in zip(labels_x.keys(), probs_x):
        total += prob * labels_entropy([y for x, y in zip(xs, ys) if x == lab])
    return total

def information_gain(xs, ys):
    return labels_entropy(ys) - entropy_given(xs, ys)

def information_gain_all(xs, ys):
    columns = list(pd.DataFrame(xs))
    res = {}
    for p in columns:
        res[p] = information_gain(ys, xs[p])
    return res

def best_question_entropy(xs, ys):
    igs = information_gain_all(xs, ys)
    if len(igs) == 0:
        print(xs)
    return max(igs, key=igs.get)

## Gini index

In [154]:
def gini(X):
    lx = len(X)
    t = sum(X) / lx
    r = pow((len(X[X > t]) / lx), 2) + pow((len(X[X <= t]) / lx), 2)
    return 1 - r

def gini_X_given_Y(X, y):
    ly = len(y)
    t = sum(y) / ly
    i = (len(y[y > t]) / ly) * gini(X[y > t])
    j = (len(y[y <= t]) / ly) * gini(X[y <= t])
    return i + j

def best_question_gini(X, y):
    indx = imp = 0
    for i in range(len(X.columns)):
        newGI = gini(y) - gini_X_given_Y(y, X[i])
        if newGI > imp:
            imp = newGI
            indx = i    
    return indx

## Helpers

In [123]:
def train_split(X, y, train_size=0.7):
    df = X.copy()
    df['y'] = y    
    train = df.sample(frac=train_size)
    test = df.drop(train.index)
    return (train.iloc[:,:-1], train.iloc[:,-1], test.iloc[:,:-1], test.iloc[:,-1])

def best_question(xs, ys, impurity_measure='entropy'):
    if impurity_measure == 'entropy':
        return best_question_entropy(xs, ys)
    else:
        return best_question_gini(xs, ys)

## Create the three objects

In [318]:
class DecitionTree:
    def __init__(self):
        self.rootNode = ""
        self.algorithm = ""
        
    def learn(self, X, y, impurity_measure='entropy', prune=False):
        self.algorithm = impurity_measure
        self.rootNode = Node(X, y, "root", self.algorithm)
        if prune:
            self.prune()

    def predict(self, X):
        res = []
        for index, rw in X.iterrows():
            res.append(self.rootNode.search(rw))
        df = pd.DataFrame()
        df["result"] = res
        return df
    
    def accuracy(self, rs, ty):
        corr = 0
        for rx, ry in zip(rs["result"], ty):
            if (rx == ry):
                corr += 1
        return corr / len(ty)
    
    def prune(self):
        return 0
    
    def toString(self):
        return self.rootNode.toString()
    

class Node:        
    def __init__(self, nX, ny, path, algo):
        self.X = nX.copy()
        self.y = ny.copy()
        self.path = path
        self.algorithm = algo
        self.result = ""
        self.child = {}
        self.question = -1
        
        if labels_entropy(self.y) == 0:
            self.isLeaf = True
            self.result = self.y.iloc[0]
        else:
            self.isLeaf = False
            Node.split(self)
    
    def split(self):
        self.question = best_question(self.X, self.y)
        i = self.question
        self.X['y'] = self.y
        for lbs in np.unique(self.X[i]):
            newTable = self.X.loc[self.X.loc[:,i] == lbs].drop(i, axis=1)
            newX = newTable.iloc[:,:-1]
            newY = newTable.iloc[:,-1]
            #print("creating node " + str(lbs))
            n = Node(newX, newY, lbs, self.algorithm)
            self.child[lbs] = n
    def search(self, rw):
        if (self.isLeaf):
            return self.result
        else:
            return self.child[rw[self.question]].search(rw)
    
    def allChildrenLeaf(self):
        for c in self.child().values():
            if not c.isLeaf:
                return False
        return True
    
    def toString(self):
        mstr = ""
        if self.isLeaf:
            return "(" + self.path + " - " + self.result + ")"
        else:
            mstr += str(self.path) + " " + str(self.question) + ": ("
            for ch in self.child.values():
                mstr += ch.toString() + " "
            mstr += ")"
            return mstr

## Insert some test data

In [323]:
import numpy as np
import pandas as pd
import random as rnd

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

data = pd.read_csv('tennis.data',header=None)
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

print(labels_entropy(y))
print(entropy_given(X[0], y))

0.9402859586706311
0.6935361388961918


In [327]:
cf = DecitionTree()

#This does NOT choose a good fit, might precent "unsolvable" problems
#Re-run if that becomes the case
lX, ly, tX, ty = train_split(X, y)
cf.learn(lX, ly)

lr = cf.predict(lX)
print(cf.accuracy(lr, ly))

tr = cf.predict(X)
print(cf.accuracy(tr, y))

1.0
0.9285714285714286


In [325]:
#Draw tree
print(cf.toString())

root 2: (high 0: ((overcast - yes) rain 3: ((strong - no) (weak - yes) ) (sunny - no) ) (normal - yes) )


## Import data

In [269]:
data = pd.read_csv('abalone.data',header=None)
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

AttributeError: 'numpy.ndarray' object has no attribute 'read_csv'

In [255]:
cf = DecitionTree()
lX, ly, tX, ty = train_split(X, y)
cf.learn(lX, ly)

lr = cf.predict(lX)
print(cf.accuracy(lr, ly))

tr = cf.predict(X)
print(cf.accuracy(tr, y))

KeyboardInterrupt: 

## Run the sklearn

In [316]:
import numpy as np
import pandas as pd
import random as rnd

import sklearn
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv('abalone.data',header=None)
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

r = np.unique(X.iloc[:,0])
dc = {}

c = 0
for l in r:
    dc[l] = c
    c += 1

X.iloc[:,0] = X.iloc[:,0].apply(lambda x: dc[x])



lX, ly, tX, ty = train_split(X, y)

cft = DecisionTreeClassifier()
cft.fit(lX,ly)
pd = cft.predict(tX)
cft.score(tX, ty)

0.19632881085395051